<a href="https://colab.research.google.com/github/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/blob/main/code/The_Food_Hazard_Detection_Challenge_SemEval_2025_The_BERT_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install torch transformers datasets pandas scikit-learn pickle openai google-generativeai anthropic numpy

In [ ]:
%%capture
!pip install anthropic

In [ ]:
%%capture
!pip install --upgrade datasets

In [ ]:
!git clone https://github.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io.git

Cloning into 'food-hazard-detection-semeval-2025.github.io'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 130 (delta 60), reused 60 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (130/130), 4.00 MiB | 18.72 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [ ]:
from transformers import BertTokenizer
import pandas as pd
data = pd.read_csv('food-hazard-detection-semeval-2025.github.io/data/incidents_train.csv', index_col=0)
data.sample()

,year,month,day,country,title,text,hazard-category,product-category,hazard,product
3760,2019,10,4,ca,Various raw beef and raw veal products recalle...,Updated Food Recall Warning - Various raw beef...,biological,"meat, egg and dairy products",escherichia coli,raw beef meat


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler, DataCollatorWithPadding, AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
from sklearn.metrics import classification_report
import pickle
from google.colab import userdata, drive
from typing_extensions import TypedDict
import json
import re
import time
import requests
import google.generativeai as genai

In [ ]:
data.title.str.split().apply(len).describe()

,title
count,5082.000000
mean,13.282369
std,5.229355
min,1.000000
25%,10.000000
50%,13.000000
75%,16.000000
max,44.000000


In [ ]:
# Count the number of examples for each class in 'product-category' and 'hazard-category'
for column in ['product-category', 'hazard-category']:
    print(f"Class counts for {column}:")
    class_counts = data[column].value_counts()
    print(class_counts)
    print("-" * 40)

Class counts for product-category:
product-category
meat, egg and dairy products                         1434
cereals and bakery products                           671
fruits and vegetables                                 535
prepared dishes and snacks                            469
seafood                                               268
soups, broths, sauces and condiments                  264
nuts, nut products and seeds                          262
ices and desserts                                     222
cocoa and cocoa preparations, coffee and tea          210
confectionery                                         170
non-alcoholic beverages                               134
dietetic foods, food supplements, fortified foods     131
herbs and spices                                      125
alcoholic beverages                                    59
other food product / mixed                             55
pet feed                                               20
fats and oils       

In [ ]:
# filter the data by hazard-category
specific_hazard_categories = ['migration', 'food additives and flavourings', 'organoleptic aspects', 'packaging defect']  # Replace with actual categories

# Filter the data to include only the rows with the specified hazard categories
filtered_data = data[data['hazard-category'].isin(specific_hazard_categories)]

# Save the filtered data to a CSV file
filtered_data.to_csv('filtered_data.csv', index=False)

In [ ]:
# filter the data by product-category
specific_product_categories = ['alcoholic beverages', 'other food product / mixed', 'pet feed', 'fats and oils', 'food additives and flavourings', 'honey and royal jelly', 'food contact materials', 'feed materials', 'sugars and syrups']  # Replace with actual categories

# Filter the data to include only the rows with the specified product categories
filtered_data = data[data['product-category'].isin(specific_product_categories)]

# Save the filtered data to a CSV file
filtered_data.to_csv('filtered_data.csv', index=False)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
synthesized_df = pd.DataFrame()
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_2.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_3.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_4.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_5.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_6.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_product.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_product_2.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_product_3.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_product_4.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])
generated_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_product_5.csv', index_col=0)
synthesized_df = pd.concat([synthesized_df, generated_data])

In [ ]:
synthesized_df['hazard'].value_counts()

,count
hazard-category,
allergens,757
chemical,702
biological,535
foreign bodies,377
other hazard,318
packaging defect,238
fraud,225
migration,217
organoleptic aspects,188


## Hazard-Category Synthesis

In [ ]:
OPENAI_API_KEY = ''
GOOGLE_API_KEY = ''
ANTHROPIC_API_KEY = ''
HF_TOKEN = ''

In [ ]:
def extract_json_from_text(response_text):
    """
    Extracts JSON block from text content returned by Gemini.
    Handles markdown blocks like ```json ... ```
    """
    try:
        # Use regex to extract JSON content wrapped in triple backticks
        match = re.search(r"```json\n(.*?)\n```", response_text, re.DOTALL)
        if match:
            json_block = match.group(1)  # Extracted JSON
            return json.loads(json_block)  # Parse to Python object
        else:
            print("No JSON block found. Attempting to parse raw text...")
            return json.loads(response_text)  # Fallback for raw JSON
    except Exception as e:
        print(f"Error extracting JSON: {e}")
        return []

In [ ]:
def extract_json_from_text2(response_text):
    """
    Extracts JSON block from text content returned by Gemini.
    Handles markdown blocks like ```json ... ```
    """
    try:
        # Use regex to extract JSON content wrapped in triple backticks
        match = re.search(r"\{.*\}", response_text, re.DOTALL)
        if match:
            json_block = match.group(1)  # Extracted JSON
            return json.loads(json_block)  # Parse to Python object
        else:
            print("No JSON block found. Attempting to parse raw text...")
            return json.loads(response_text)  # Fallback for raw JSON
    except Exception as e:
        print(f"Error extracting JSON: {e}")
        return []

In [ ]:
import google.generativeai as genai

# Number of samples per category (can be easily changed later)
SAMPLE_SIZE = 1000

# Get the 6 hazard-categories with the least samples
# least_common_categories = data['hazard-category'].value_counts().nsmallest(6).index
least_common_categories = ['fraud']

# Collect the required samples for each category
sampled_dataframes = []
for category in least_common_categories:
    category_data = data[data['hazard-category'] == category]

    if len(category_data) < SAMPLE_SIZE:
        # Duplicate samples if fewer than SAMPLE_SIZE
        multiplier = -(-SAMPLE_SIZE // len(category_data))  # Ceiling division
        category_data = pd.concat([category_data] * multiplier, ignore_index=True)

    sampled_dataframes.append(category_data.sample(SAMPLE_SIZE, random_state=52))

# Combine all sampled data into one dataframe
final_dataframe = pd.concat(sampled_dataframes, ignore_index=True)

In [ ]:
final_dataframe = pd.read_csv('gemini_augmented_5.csv', index_col=0)
final_dataframe.sample()

,month,day,country,title,text,hazard-category,product-category,hazard,product
year,,,,,,,,,
2018,11,18,au,Konjac Jelly Recall - Potential Choking Hazard,"Merry Seasons Kiwi Flavour Mini Jelly Cups, 36...",food additives and flavourings,confectionery,e 425 - konjac unauthorised,jellies


In [ ]:
from tqdm import tqdm  # Import tqdm for the progress bar
import pandas as pd
import time
import json  # For handling JSON decoding

# Configure Gemini model
class IncidentReport(TypedDict):
    title: str
    text: str

genai.configure(api_key='')
model = genai.GenerativeModel("gemini-1.5-flash-8b")

def generate_new_row(row):
    prompt = f"""
    Generate a new incident report based on the following details. Ensure the generated 'title' and 'text' are as different as possible from the input 'title' and 'text'. Only return the 'title' and 'text' in valid JSON format:
    Title: {row['title']}
    Text: {row['text']}
    Hazard Category: {row['hazard-category']}
    Product Category: {row['product-category']}
    Hazard: {row['hazard']}
    Product: {row['product']}
    """
    response = model.generate_content(prompt)

    try:
        # Parse the JSON response
        generated_data = extract_json_from_text(response.text)

        # Validate required fields
        if 'title' not in generated_data or 'text' not in generated_data:
            raise ValueError("Missing 'title' or 'text' in the response")

        # Return a complete row with all original columns, replacing title and text
        new_row = row.to_dict()
        new_row['title'] = generated_data['title']
        new_row['text'] = generated_data['text']
        return new_row

    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error parsing JSON for row: {row}. Error: {e}")
        return None  # Skip this row if parsing fails

# Generate new rows using Gemini and add them to a new dataframe
new_rows = []
for index, row in tqdm(final_dataframe.iterrows(), total=len(final_dataframe), desc="Augmenting rows"):
    new_row = generate_new_row(row)
    if new_row is not None:
        new_rows.append(new_row)
    time.sleep(5)  # Simulate delay or API rate limit

# Convert the list of new rows to a DataFrame
generated_dataframe = pd.DataFrame(new_rows)

# Save the generated dataframe to a CSV file
generated_dataframe.to_csv('gemini_augmented_fraud.csv', index=False)

# Display the generated dataframe
print("Augmentation complete.")
generated_dataframe


Augmenting rows:  37%|███▋      | 369/1000 [47:14<1:25:56,  8.17s/it]

Error extracting JSON: Invalid control character at: line 3 column 327 (char 394)
Error parsing JSON for row: year                                                             2019
month                                                               5
day                                                                15
country                                                            us
title               Crab House Trading Corp. Recalls Siluriformes ...
text                EDITOR’S NOTE:Details of this recall were upda...
hazard-category                                                 fraud
product-category                                              seafood
hazard                                              inspection issues
product                                          frozen fish products
Name: 369, dtype: object. Error: Missing 'title' or 'text' in the response


Augmenting rows:  60%|█████▉    | 596/1000 [1:26:01<58:18,  8.66s/it]


ReadTimeout: HTTPConnectionPool(host='localhost', port=43173): Read timed out. (read timeout=600.0)

In [ ]:
from huggingface_hub import login
login(token="REMOVED")

In [ ]:
# Set your Hugging Face API token
HF_API_TOKEN = "REMOVED"

In [ ]:
# Define the endpoint and headers
API_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-2-7b-chat-hf"
headers = {"Authorization": f"Bearer {HF_API_TOKEN}"}

# Define the function to generate a new row
def generate_new_row(row):
    prompt = f"""
    Generate a new incident report based on the following details. Ensure the generated 'title' and 'text' are as different as possible from the input 'title' and 'text'. Only return the 'title' and 'text' in valid JSON format:
    Title: {row['title']}
    Text: {row['text']}
    Hazard Category: {row['hazard-category']}
    Product Category: {row['product-category']}
    Hazard: {row['hazard']}
    Product: {row['product']}
    """
    payload = {"inputs": prompt, "parameters": {"max_length": 512, "temperature": 0.7}}
    response = requests.post(API_URL, headers=headers, json=payload)

    if response.status_code == 200:
        response_text = response.text
        print(response_text)
        try:
            # Parse the JSON response
            generated_data = json.loads(response_text)

            # Validate required fields
            if 'title' not in generated_data or 'text' not in generated_data:
                raise ValueError("Missing 'title' or 'text' in the response")

            # Return a complete row with all original columns, replacing title and text
            new_row = row.to_dict()
            new_row['title'] = generated_data['title']
            new_row['text'] = generated_data['text']
            return new_row

        except (json.JSONDecodeError, ValueError) as e:
            print(f"Error parsing JSON for row: {row}. Error: {e}")
            return None  # Skip this row if parsing fails
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Assuming 'final_dataframe' is your input DataFrame
new_rows = []
for _, row in final_dataframe.iterrows():
    new_row = generate_new_row(row)
    if new_row is not None:
        new_rows.append(new_row)

generated_dataframe = pd.DataFrame(new_rows)

# Save the generated dataframe to a CSV file
generated_dataframe.to_csv('llama_augmented.csv', index=False)

# Display the generated dataframe
generated_dataframe


In [ ]:
# Load the Llama 3.2 model and tokenizer from Hugging Face
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the function to generate a new row
def generate_new_row(row):
    prompt = f"""
    Generate a new incident report based on the following details. Ensure the generated 'title' and 'text' are as different as possible from the input 'title' and 'text'. Only return the 'title' and 'text' in valid JSON format.
    {{
      "Title": "{row['title']}",
      "Text": "{row['text']}",
      "Hazard Category": "{row['hazard-category']}",
      "Product Category": "{row['product-category']}",
      "Hazard": "{row['hazard']}",
      "Product": "{row['product']}"
    }}
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=1024, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Generated Response:\n", response_text)

    try:
        # Extract JSON object using regex
        json_match = re.search(r"\{.*\}", response_text, re.DOTALL)
        if not json_match:
            raise ValueError("No valid JSON object found in the response")

        generated_data = json.loads(json_match.group())

        # Validate required fields
        if 'title' not in generated_data or 'text' not in generated_data:
            raise ValueError("Missing 'title' or 'text' in the response")

        # Return a complete row with all original columns, replacing title and text
        new_row = row.to_dict()
        new_row['title'] = generated_data['title']
        new_row['text'] = generated_data['text']
        return new_row

    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error parsing JSON for row: {row}. Error: {e}")
        return None  # Skip this row if parsing fails

# Assuming 'final_dataframe' is your input DataFrame
new_rows = []
for _, row in final_dataframe.iterrows():
    new_row = generate_new_row(row)
    if new_row is not None:
        new_rows.append(new_row)
    time.sleep(5)

# Convert the list of new rows into a DataFrame
generated_dataframe = pd.DataFrame(new_rows)

# Save the generated DataFrame to a CSV file
generated_dataframe.to_csv('llama_augmented.csv', index=False)

# Display the generated DataFrame
generated_dataframe

In [ ]:
# Load the Mistral-7B-Instruct-v0.3 model and tokenizer from Hugging Face
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the function to generate a new row
def generate_new_row(row):
    prompt = f"""
    Generate a new incident report based on the following details. Ensure the generated 'title' and 'text' are as different as possible from the input 'title' and 'text'. Only return the 'title' and 'text' in valid JSON format:
    Title: {row['title']}
    Text: {row['text']}
    Hazard Category: {row['hazard-category']}
    Product Category: {row['product-category']}
    Hazard: {row['hazard']}
    Product: {row['product']}
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=512, temperature=0.7)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response_text)

    try:
        # Parse the JSON response
        generated_data = json.loads(response_text)

        # Validate required fields
        if 'title' not in generated_data or 'text' not in generated_data:
            raise ValueError("Missing 'title' or 'text' in the response")

        # Return a complete row with all original columns, replacing title and text
        new_row = row.to_dict()
        new_row['title'] = generated_data['title']
        new_row['text'] = generated_data['text']
        return new_row

    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error parsing JSON for row: {row}. Error: {e}")
        return None  # Skip this row if parsing fails

# Assuming 'final_dataframe' is your input DataFrame
new_rows = []
for _, row in final_dataframe.iterrows():
    new_row = generate_new_row(row)
    if new_row is not None:
        new_rows.append(new_row)

generated_dataframe = pd.DataFrame(new_rows)

# Save the generated dataframe to a CSV file
generated_dataframe.to_csv('mistral_augmented.csv', index=False)

# Display the generated dataframe
generated_dataframe

In [ ]:
# Define the endpoint and headers
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"
headers = {"Authorization": f"Bearer {HF_API_TOKEN}"}

# Define the function to generate a new row
def generate_new_row(row):
    prompt = f"""
    Generate a new incident report based on the following details. Ensure the generated 'title' and 'text' are as different as possible from the input 'title' and 'text'. Only return the 'title' and 'text' in valid JSON format.

    Details:
    Title: {row['title']}
    Text: {row['text']}
    Hazard Category: {row['hazard-category']}
    Product Category: {row['product-category']}
    Hazard: {row['hazard']}
    Product: {row['product']}
    """
    payload = {"inputs": prompt, "parameters": {"max_length": 2048, "temperature": 0.7}}
    response = requests.post(API_URL, headers=headers, json=payload)

    if response.status_code == 200:
        response_text = response.json()[0]
        print(response_text)
        print(response_text['generated_text'])
        generated_data = extract_json_from_text2(response_text['generated_text'])
        print(generated_data)
        print(response_text['generated_text'][-10:-1])

        print(response_text['text'])

        # Parse the JSON response
        generated_data = json.loads(response_text)

        # Validate required fields
        if 'title' not in generated_data or 'text' not in generated_data:
            raise ValueError("Missing 'title' or 'text' in the response")

        # Return a complete row with all original columns, replacing title and text
        new_row = row.to_dict()
        new_row['title'] = generated_data['title']
        new_row['text'] = generated_data['text']
        return new_row

# Assuming 'final_dataframe' is your input DataFrame
new_rows = []
for _, row in final_dataframe.iterrows():
    new_row = generate_new_row(row)
    if new_row is not None:
        new_rows.append(new_row)

generated_dataframe = pd.DataFrame(new_rows)

# Save the generated dataframe to a CSV file
generated_dataframe.to_csv('mistral_augmented.csv', index=False)

# Display the generated dataframe
generated_dataframe

In [ ]:
# Load Llama 2 model and tokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def generate_new_row(row):
    prompt = f"""
    Generate a new incident report based on the following details. Ensure the generated 'title' and 'text' are as different as possible from the input 'title' and 'text'. Only return the 'title' and 'text' in valid JSON format:
    Title: {row['title']}
    Text: {row['text']}
    Hazard Category: {row['hazard-category']}
    Product Category: {row['product-category']}
    Hazard: {row['hazard']}
    Product: {row['product']}
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=512, temperature=0.7)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response_text)

    try:
        # Parse the JSON response
        generated_data = json.loads(response_text)
        if 'title' not in generated_data or 'text' not in generated_data:
            raise ValueError("Missing 'title' or 'text' in the response")
        new_row = row.to_dict()
        new_row['title'] = generated_data['title']
        new_row['text'] = generated_data['text']
        return new_row
    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error parsing JSON for row: {row}. Error: {e}")
        return None  # Skip this row if parsing fails

# Process the DataFrame
new_rows = []
for _, row in final_dataframe.iterrows():
    new_row = generate_new_row(row)
    if new_row is not None:
        new_rows.append(new_row)

generated_dataframe = pd.DataFrame(new_rows)

# Save the results
generated_dataframe.to_csv('llama2_augmented.csv', index=False)
print(generated_dataframe)


## Product-Category Synthesis

In [ ]:
def extract_json_from_text(response_text):
    """
    Extracts JSON block from text content returned by Gemini.
    Handles markdown blocks like ```json ... ```
    """
    try:
        # Use regex to extract JSON content wrapped in triple backticks
        match = re.search(r"```json\n(.*?)\n```", response_text, re.DOTALL)
        if match:
            json_block = match.group(1)  # Extracted JSON
            return json.loads(json_block)  # Parse to Python object
        else:
            print("No JSON block found. Attempting to parse raw text...")
            return json.loads(response_text)  # Fallback for raw JSON
    except Exception as e:
        print(f"Error extracting JSON: {e}")
        return []

In [ ]:
import google.generativeai as genai

# Number of samples per category (can be easily changed later)
SAMPLE_SIZE = 10

# Get the 6 product-categories with the least samples
least_common_categories = data['product-category'].value_counts().nsmallest(15).index

# Collect the required samples for each category
sampled_dataframes = []
for category in least_common_categories:
    category_data = data[data['product-category'] == category]

    if len(category_data) < SAMPLE_SIZE:
        # Duplicate samples if fewer than SAMPLE_SIZE
        multiplier = -(-SAMPLE_SIZE // len(category_data))  # Ceiling division
        category_data = pd.concat([category_data] * multiplier, ignore_index=True)

    sampled_dataframes.append(category_data.sample(SAMPLE_SIZE, random_state=32))

# Combine all sampled data into one dataframe
final_dataframe = pd.concat(sampled_dataframes, ignore_index=True)

In [ ]:
final_dataframe = pd.read_csv('gemini_augmented_product_3.csv', index_col=0)
final_dataframe.sample()

,month,day,country,title,text,hazard-category,product-category,hazard,product
year,,,,,,,,,
2015,9,30,au,Urgent Recall: Undeclared Dairy in Sports Supp...,A recall is underway for multiple protein powd...,fraud,"dietetic foods, food supplements, fortified foods",dairy products,dietary supplement


In [ ]:
final_dataframe.to_csv('sampled_data_for_augmentation_product.csv')

In [ ]:
from google.colab import files
# files.download('sampled_data_for_augmentation_product.csv')

In [ ]:
final_dataframe['product-category'].value_counts()

,count
product-category,
sugars and syrups,50
feed materials,50
food contact materials,50
food additives and flavourings,50
honey and royal jelly,50
fats and oils,50
pet feed,50
other food product / mixed,50
alcoholic beverages,50


In [ ]:
# Configure Gemini model
class IncidentReport(TypedDict):
    title: str
    text: str

genai.configure(api_key='')
model = genai.GenerativeModel("gemini-1.5-flash-8b")

def generate_new_row(row):
    prompt = f"""
    Generate a new incident report based on the following details. Ensure the generated 'title' and 'text' are as different as possible from the input 'title' and 'text'. Only return the 'title' and 'text' in valid JSON format:
    Title: {row['title']}
    Text: {row['text']}
    Hazard Category: {row['hazard-category']}
    Product Category: {row['product-category']}
    Hazard: {row['hazard']}
    Product: {row['product']}
    """
    response = model.generate_content(
        prompt,
        # generation_config=genai.GenerationConfig(
        #     response_mime_type="application/json",
        #     response_schema=IncidentReport,
        #     temperature=0.7
        # )
    )

    print(response.text)

    try:
        # Parse the JSON response
        generated_data = extract_json_from_text(response.text)

        # Validate required fields
        if 'title' not in generated_data or 'text' not in generated_data:
            raise ValueError("Missing 'title' or 'text' in the response")

        # Return a complete row with all original columns, replacing title and text
        new_row = row.to_dict()
        new_row['title'] = generated_data['title']
        new_row['text'] = generated_data['text']
        return new_row

    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error parsing JSON for row: {row}. Error: {e}")
        return None  # Skip this row if parsing fails

# Generate new rows using Gemini and add them to a new dataframe
new_rows = []
for index, row in final_dataframe.iterrows():
    new_row = generate_new_row(row)
    if new_row is not None:
        new_rows.append(new_row)
    time.sleep(5)

generated_dataframe = pd.DataFrame(new_rows)

# Save the generated dataframe to a CSV file
generated_dataframe.to_csv('gemini_augmented_product_5.csv', index=False)

# Display the generated dataframe
generated_dataframe


```json
{
  "title": "Coconut Sugar Recall Due to Undeclared Gluten",
  "text": "Urgent Recall Notice: Intenson Bio's Organic Coconut Sugar, potentially containing undeclared gluten, is being recalled due to a manufacturing error.  Consumers with gluten sensitivities are urged to check affected batches listed below.  This recall impacts batches from [Date range, if available].  Contact [Company contact info] for further information and a refund or exchange.  This product could pose a serious health risk to those with celiac disease or gluten intolerance.  Report any adverse reaction to [regulatory agency contact]."
}
```

```json
{
  "title": "Grocery Chain Recall: Metal Fragments Found in Sugar Used for Prepared Meals",
  "text": "A recall has been initiated for several brands of prepared meals sold in grocery stores across the nation, after the discovery of potential metal fragments within the sugar used in the sauces.  The affected products, distributed by a major food processing co

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-8b:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 685.00ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-8b:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 912.73ms


```json
{
  "title": "Urgent Recall of Plastic Food Contact Materials Due to Unregistered Facility",
  "text": "Immediate recall of all batches of Hugold plastic food contact materials.  These products, originating from Ireland, were processed in an unregistered facility lacking proper food safety and hygiene protocols.  The presence of concentrated, unapproved material poses a serious health risk and violates EU regulations.  Specifically, Article 6.2 of Regulation EU 2015/2283 mandates thorough assessment of novel food and its related products for market suitability.  This recall spans all batches with all best-before dates. Consumers are strongly advised not to use any implicated products.  Further details are available upon request. Report reference 2022.07"
}
```

```json
{
  "title": "Hilly Acres Farm Varnish Recall - Potential Fraud Concerns",
  "text": "Urgent Recall Notice Regarding Hilly Acres Farm Products - Potential Contamination of Food Contact Materials.  The Canadian Fo

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-8b:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 710.37ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-8b:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 734.58ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-8b:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 685.77ms


```json
{
  "title": "Tofu Recall Due to Bacterial Contamination in Western Australia",
  "text": "Multiple varieties of tofu produced by Aus Soya Products are being recalled due to confirmed presence of E. coli and Listeria monocytogenes.  The recall affects five different product types, including fried, soft, and cutlet varieties, with varying weights and expiration dates.  Consumers in Western Australia are urged to return affected products to their point of purchase for a full refund.  The recall is coordinated by Food Standards Australia New Zealand.  Individuals with compromised immune systems, pregnant women, young children, and the elderly are advised to exercise particular caution.  Detailed product information, including specific product codes and expiration dates, can be found on the Food Standards Australia New Zealand website.  The hazard category is biological, affecting various tofu products.  The affected products were sold in Western Australia."
}
```

```json
{
  "tit

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-8b:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2152.71ms


```json
{
  "title": "Soylent Recalls Lot of Protein Powder Due to Potential Milk Contamination",
  "text": "Soylent is recalling a specific batch of its protein powder, Soylent 1.8, due to a potential contamination concern.  A small amount of whey protein may have been inadvertently mixed into the production line, posing a risk to individuals with milk allergies. The affected batch, lot number G7076PA with an expiration date of February 2018, is being recalled.  Customers who possess any Soylent 1.8 powder from this lot are urged to immediately discard the product.  The affected product (SKU: 1WK-V108) is packaged in 7 x 15oz pouches and 890 boxes have been identified as potentially contaminated. Soylent is contacting impacted customers directly.  No reported illnesses are associated with this recall, and future Soylent 1.8 powder batches are not affected.  A full refund or replacement is available for affected customers who can contact Soylent at info@soylent.com for details.  The re

,year,month,day,country,title,text,hazard-category,product-category,hazard,product
0,2020,2,24,ie,Coconut Sugar Recall Due to Undeclared Gluten,Urgent Recall Notice: Intenson Bio's Organic C...,allergens,sugars and syrups,cereals containing gluten and products thereof,coconut sugar
1,2019,4,30,us,Grocery Chain Recall: Metal Fragments Found in...,A recall has been initiated for several brands...,foreign bodies,sugars and syrups,metal fragment,sugar
2,2019,4,30,us,Retail Recall Expands: Potential Metal Fragmen...,A broader recall is now underway for certain f...,foreign bodies,sugars and syrups,metal fragment,sugar
3,2016,7,19,us,Recall Expanded for P.F. Chang's Home Menu Pro...,ConAgra Foods is expanding a voluntary recall ...,foreign bodies,sugars and syrups,metal fragment,sugar
4,2022,5,27,ca,Plastic Fragments Found in Rogers Sugar - Urge...,Rogers brand Golden Yellow Sugar is under imme...,foreign bodies,sugars and syrups,plastic fragment,sugar
...,...,...,...,...,...,...,...,...,...,...
1419,2004,2,10,au,Azzura Gelato Recall: Potential Nut Allergen,A recent investigation into a batch of Azzura ...,allergens,ices and desserts,other not classified allergen hazards,ice cream
1420,2020,1,24,uk,Urgent Recall of Berrylicious Yogurt Parfaits ...,Berrylicious Yogurt Parfaits are being recalle...,allergens,ices and desserts,milk and products thereof,pudding
1421,2008,2,20,us,Pierre's Ice Cream Recall: Potential Undeclare...,Pierre's Ice Cream is recalling a batch of Van...,allergens,ices and desserts,peanuts and products thereof,ice cream
1422,2022,4,12,ie,Incident Report: Ethylene Oxide Contamination...,"On October 26, 2023, a batch of artisanal choc...",chemical,ices and desserts,unauthorised substance ethylene oxide,ice cream


In [ ]:
files.download('gemini_augmented_product_5.csv')
generated_dataframe.to_csv('/content/drive/My Drive/FoodHazardData/Gemini/gemini_augmented_product_5.csv', index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import pipeline
import time
import pandas as pd

# Load the lightweight T5 model for text generation
generator = pipeline("text2text-generation", model="t5-small", device=-1)  # Use `device=-1` for CPU

def generate_new_row_t5_light(row):
    prompt = f"""
    Generate a new incident report based on the following details.
    {row['title']}
    {row['text']}
    {row['hazard-category']}
    {row['product-category']}
    {row['hazard']}
    {row['product']}
    """
    # try:
    response = generator(prompt, max_length=2048, num_return_sequences=1, temperature=0.7)
    print(response)
    generated_text = response[0]['generated_text']
    print(generated_text)

    # Extract JSON-like data from the generated text
    generated_data = generated_text

    # Validate required fields
    # if 'title' not in generated_data or 'text' not in generated_data:
    #     raise ValueError("Missing 'title' or 'text' in the response")

    # Return a complete row with all original columns, replacing title and text
    new_row = row.to_dict()
    # new_row['title'] = generated_data['title']
    new_row['text'] = generated_text
    return new_row

    # except Exception as e:
    #     print(f"Error generating or parsing JSON for row: {row}. Error: {e}")
    #     return None  # Skip this row if parsing fails

# Generate new rows using lightweight T5 and add them to a new dataframe
new_rows = []
for index, row in final_dataframe.iterrows():
    new_row = generate_new_row_t5_light(row)
    if new_row is not None:
        new_rows.append(new_row)
    # time.sleep(2)  # Adjust as needed

generated_dataframe_t5_light = pd.DataFrame(new_rows)

# Save the generated dataframe to a CSV file
generated_dataframe_t5_light.to_csv('t5_augmented_product.csv', index=False)

# Display the generated dataframe
generated_dataframe_t5_light

Device set to use cpu
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1322 > 512). Running this sequence through the model will result in indexing errors


[{'generated_text': 'Canada.ca Notification Rogers brand Golden Yellow Sugar recalled due to pieces of plastic Brand(s) Rogers Last updated 2022-05-24 Summary Product Golden Yellow Sugar Issue Food - Extraneous Material What to do Do not use, sell, serve or distribute the affected product . The recalled product was distributed in Alberta and British Columbia. Additional information Details Original published date: 2022-05-27 Alert / recall type Notification Category Food - Candy, confectionary, snacks and sweeteners Companies Lantic Inc.'}]
Canada.ca Notification Rogers brand Golden Yellow Sugar recalled due to pieces of plastic Brand(s) Rogers Last updated 2022-05-24 Summary Product Golden Yellow Sugar Issue Food - Extraneous Material What to do Do not use, sell, serve or distribute the affected product . The recalled product was distributed in Alberta and British Columbia. Additional information Details Original published date: 2022-05-27 Alert / recall type Notification Category Foo

## Rest Code

In [ ]:
data['title_text'] = data['title'] + ' ' + data['text']
# synthesized_df['title_text'] = synthesized_df['title'] + ' ' + synthesized_df['text']

# data['title_text'] = 'Hazard Category: ' + data['hazard-category'] + ' ' + data['title'] + ' ' + data['text']
# synthesized_df['title_text'] = 'Hazard Category: ' + synthesized_df['hazard-category'] + ' ' + synthesized_df['title'] + ' ' + synthesized_df['text']

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer.save_pretrained('./bert_tokenizer')
# tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer.save_pretrained('./tokenizer')
# tokenizer.save_pretrained('./distilbert_tokenizer')
# tokenizer.save_pretrained('/content/drive/My Drive/FoodHazardData/tokenizer')

def tokenize_function(examples):
    return tokenizer(examples['title_text'], padding=True, truncation=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Label: `Hazard Category`

* Choose your target

In [ ]:
label = 'hazard-category' # change this to: 'product-category', 'hazard', 'product' to alter the ground truth
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data[label])
# synthesized_df['label'] = label_encoder.transform(synthesized_df[label])

with open(f'{label}_label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [ ]:
train_df, test_df = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
from nltk.corpus import wordnet
import random
import nltk

# Download required NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')

# ----------------------------
# 1. Synonym Replacement Functions
# ----------------------------
def get_synonyms(word):
    """Retrieve synonyms for a given word using WordNet."""
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ').lower()
            if synonym != word.lower():
                synonyms.add(synonym)
    return list(synonyms)

def synonym_replacement(sentence, n=1):
    """
    Replace up to n words in the sentence with their synonyms.

    Args:
        sentence (str): The original sentence.
        n (int): Number of words to replace.

    Returns:
        str: The augmented sentence.
    """
    words = sentence.split()
    new_words = words.copy()
    eligible_words = [word for word in words if word.isalpha()]
    random.shuffle(eligible_words)
    num_replaced = 0

    for random_word in eligible_words:
        synonyms = get_synonyms(random_word)
        if synonyms:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    return ' '.join(new_words)

# ----------------------------
# 2. Data Splitting
# ----------------------------
# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.1, random_state=42)
train_df = pd.concat([train_df, synthesized_df], ignore_index=True)


# ----------------------------
# 3. Augment the Training Data
# ----------------------------
# augmented_data = []

# # 3a. Class counts and thresholds for augmentation
# class_counts = train_df['hazard-category'].value_counts()
# minority_threshold = 310   # Classes below this threshold will be augmented
# augmentations_per_class = 150  # Number of new samples to add for any minority class
# minimum_class_size = 150   # Ensure each class is at least this big if it's too small
# minority_classes = class_counts[class_counts < minority_threshold].index.tolist()

# # ----------------------------
# # 3b. Define a helper for augmentation
# # ----------------------------
# def augment_class_data(class_df, target_total):
#     """
#     Augment `class_df` until its total size (original + new) is `target_total`.
#     Distribute augmentations across existing rows.
#     """
#     augmented_rows = []
#     current_count = len(class_df)
#     # How many new samples do we need?
#     additional_needed = target_total - current_count
#     if additional_needed <= 0:
#         return []  # No augmentation needed

#     # Distribute 'additional_needed' among the existing samples
#     times_each = additional_needed // current_count
#     remainder = additional_needed % current_count

#     # Shuffle indices for distributing that remainder randomly
#     indices = class_df.index.tolist()
#     random.shuffle(indices)

#     # For each row in the class, augment it 'times_each' times,
#     # plus 1 extra if you're within the remainder range.
#     for i, row_idx in enumerate(indices):
#         row = class_df.loc[row_idx]
#         # Determine how many times this row will be augmented
#         n_augmentations = times_each + (1 if i < remainder else 0)

#         for _ in range(n_augmentations):
#             # You can tune n=2, n=10, etc. for the number of words replaced
#             augmented_title_text = synonym_replacement(row['title_text'], n=2)
#             # augmented_text = synonym_replacement(row['text'], n=2)  # If you want to augment the other field too

#             new_row = row.copy()
#             new_row['title_text'] = augmented_title_text
#             # new_row['text'] = augmented_text
#             augmented_rows.append(new_row)

#     return augmented_rows

# # ----------------------------
# # 3c. Augment each minority class
# # ----------------------------
# for minority_class in minority_classes:
#     print(f"Augmenting class '{minority_class}'")
#     class_data = train_df[train_df['hazard-category'] == minority_class]

#     # 1) First, if it has fewer than 150 samples, bring it up to 150
#     if len(class_data) < minimum_class_size:
#         to_150 = augment_class_data(class_data, target_total=minimum_class_size)
#         augmented_data.extend(to_150)
#         # Update the class_data to include newly augmented rows so total goes up to 150
#         if to_150:
#             new_df = pd.DataFrame(to_150)
#             class_data = pd.concat([class_data, new_df], ignore_index=True)

#     # 2) Now, add an extra 'augmentations_per_class' beyond the *new* count
#     updated_count = len(class_data)
#     final_target = updated_count + augmentations_per_class
#     additional_aug = augment_class_data(class_data, target_total=final_target)
#     augmented_data.extend(additional_aug)

# # ----------------------------
# # 3d. Combine everything
# # ----------------------------
# augmented_df = pd.DataFrame(augmented_data)
# train_df = pd.concat([train_df, augmented_df], ignore_index=True)
print(f"Final augmented training set size: {len(train_df)}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Final augmented training set size: 8272


In [ ]:
train_df['hazard-category'].value_counts()

,count
hazard-category,
allergens,2423
biological,2105
chemical,954
foreign bodies,880
fraud,568
other hazard,437
packaging defect,287
organoleptic aspects,238
migration,219


* Data preprocessing

In [ ]:
# Split the data into training and testing sets
# train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create DataCollator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)

# Convert dataset to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/4573 [00:00<?, ? examples/s]

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

* Choose your model

In [ ]:
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))
model = DistilBertForSequenceClassification.from_pretrained('microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract', num_labels=len(data[label].unique()))
model.to('cuda')  # Move model to GPU if available

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/FoodHazardData/Models/distilbert_hazard_category_gemini_nlpaug and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([10]) in the checkpoint and torch.Size([22]) in the model instantiated
- classifier.weight: found shape torch.Size([10, 768]) in the checkpoint and torch.Size([22, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


* Train it

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

model.train()

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5170 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


RuntimeError: The expanded size of the tensor (17441) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [8, 17441].  Tensor sizes: [1, 512]

* Assess it

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/FoodHazardData/Models/distilbert_hazard_category_gemini_nlpaug")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.eval()
total_predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        total_predictions.extend([p.item() for p in predictions])

#print(classification_report(test_df.label, total_predictions))
predicted_labels = label_encoder.inverse_transform(total_predictions)
gold_labels = label_encoder.inverse_transform(test_df.label.values)
print(classification_report(gold_labels, predicted_labels, zero_division=0))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model.save_pretrained("bert_hazard_category_gemini_pubmed")
model.save_pretrained("/content/drive/My Drive/FoodHazardData/Models/bert_hazard_category_gemini_pubmed")

In [ ]:
import numpy as np

# List of model paths saved in Google Drive
model_paths = [
    "/content/drive/My Drive/FoodHazardData/bert_hazard_category",
    "/content/drive/My Drive/FoodHazardData/Models/bert_hazard_category_gemini",
    # "/content/drive/My Drive/FoodHazardData/bert_hazard_category_model_3"
]

# Load all models
models = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for path in model_paths:
    model = BertForSequenceClassification.from_pretrained(path)
    model.to(device)
    model.eval()
    models.append(model)

# Evaluate with ensembling
total_predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        # Move batch to GPU if available
        batch = {k: v.to(device) for k, v in batch.items()}

        # Get logits from all models
        logits_list = []
        for model in models:
            outputs = model(**batch)
            logits_list.append(outputs.logits.cpu().numpy())

        # Ensemble predictions by averaging logits
        avg_logits = np.mean(logits_list, axis=0)  # Average logits from all models

        # Get final predictions by taking the argmax of averaged logits
        predictions = np.argmax(avg_logits, axis=-1)
        total_predictions.extend(predictions)

# Decode predicted labels and gold labels
predicted_labels = label_encoder.inverse_transform(total_predictions)
gold_labels = label_encoder.inverse_transform(test_df.label.values)

# Print classification report
print(classification_report(gold_labels, predicted_labels, zero_division=0))


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


                                precision    recall  f1-score   support

                     allergens       0.96      0.99      0.98       188
                    biological       0.99      0.99      0.99       171
                      chemical       0.94      0.97      0.96        35
food additives and flavourings       1.00      0.40      0.57         5
                foreign bodies       1.00      0.98      0.99        58
                         fraud       0.88      0.82      0.85        28
                     migration       1.00      1.00      1.00         1
          organoleptic aspects       1.00      1.00      1.00         3
                  other hazard       0.82      0.93      0.88        15
              packaging defect       0.67      0.40      0.50         5

                      accuracy                           0.97       509
                     macro avg       0.93      0.85      0.87       509
                  weighted avg       0.97      0.97      0.96 

# Label: `Product Category`

In [ ]:
label = 'product-category'
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data[label])
# synthesized_df['label'] = label_encoder.transform(synthesized_df[label])

with open(f'{label}_label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [ ]:
train_df, test_df = train_test_split(data, test_size=0.1, random_state=42)
# train_df = pd.concat([train_df, synthesized_df], ignore_index=True)
train_df['product-category'].value_counts()

,count
product-category,
"meat, egg and dairy products",1303
cereals and bakery products,617
fruits and vegetables,479
prepared dishes and snacks,418
"soups, broths, sauces and condiments",243
seafood,240
"nuts, nut products and seeds",225
ices and desserts,189
"cocoa and cocoa preparations, coffee and tea",181


In [ ]:
import random
import nltk
from nltk.corpus import wordnet
import pandas as pd
from sklearn.model_selection import train_test_split

# Download required NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')

# ----------------------------
# 1. Synonym Replacement Functions
# ----------------------------
def get_synonyms(word):
    """Retrieve synonyms for a given word using WordNet."""
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ').lower()
            if synonym != word.lower():
                synonyms.add(synonym)
    return list(synonyms)

def synonym_replacement(sentence, n=1):
    """
    Replace up to n words in the sentence with their synonyms.

    Args:
        sentence (str): The original sentence.
        n (int): Number of words to replace.

    Returns:
        str: The augmented sentence.
    """
    words = sentence.split()
    new_words = words.copy()
    eligible_words = [word for word in words if word.isalpha()]
    random.shuffle(eligible_words)
    num_replaced = 0

    for random_word in eligible_words:
        synonyms = get_synonyms(random_word)
        if synonyms:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    return ' '.join(new_words)

# ----------------------------
# 2. Data Splitting
# ----------------------------
# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.1, random_state=42)
train_df = pd.concat([train_df, synthesized_df], ignore_index=True)

# ----------------------------
# 3. Augment the Training Data
# ----------------------------
# augmented_data = []

# # 3a. Class counts and thresholds for augmentation
# class_counts = train_df['product-category'].value_counts()
# minority_threshold = 351   # Classes below this threshold will be augmented
# augmentations_per_class = 150  # Number of new samples to add for any minority class
# minimum_class_size = 150   # Ensure each class is at least this big if it's too small
# minority_classes = class_counts[class_counts < minority_threshold].index.tolist()

# # ----------------------------
# # 3b. Define a helper for augmentation
# # ----------------------------
# def augment_class_data(class_df, target_total):
#     """
#     Augment `class_df` until its total size (original + new) is `target_total`.
#     Distribute augmentations across existing rows.
#     """
#     augmented_rows = []
#     current_count = len(class_df)
#     # How many new samples do we need?
#     additional_needed = target_total - current_count
#     if additional_needed <= 0:
#         return []  # No augmentation needed

#     # Distribute 'additional_needed' among the existing samples
#     times_each = additional_needed // current_count
#     remainder = additional_needed % current_count

#     # Shuffle indices for distributing that remainder randomly
#     indices = class_df.index.tolist()
#     random.shuffle(indices)

#     # For each row in the class, augment it 'times_each' times,
#     # plus 1 extra if you're within the remainder range.
#     for i, row_idx in enumerate(indices):
#         row = class_df.loc[row_idx]
#         # Determine how many times this row will be augmented
#         n_augmentations = times_each + (1 if i < remainder else 0)

#         for _ in range(n_augmentations):
#             # You can tune n=2, n=10, etc. for the number of words replaced
#             augmented_title_text = synonym_replacement(row['title_text'], n=2)
#             # augmented_text = synonym_replacement(row['text'], n=2)  # If you want to augment the other field too

#             new_row = row.copy()
#             new_row['title_text'] = augmented_title_text
#             # new_row['text'] = augmented_text
#             augmented_rows.append(new_row)

#     return augmented_rows

# # ----------------------------
# # 3c. Augment each minority class
# # ----------------------------
# for minority_class in minority_classes:
#     print(f"Augmenting class '{minority_class}'")
#     class_data = train_df[train_df['product-category'] == minority_class]

#     # 1) First, if it has fewer than 150 samples, bring it up to 150
#     if len(class_data) < minimum_class_size:
#         to_150 = augment_class_data(class_data, target_total=minimum_class_size)
#         augmented_data.extend(to_150)
#         # Update the class_data to include newly augmented rows so total goes up to 150
#         if to_150:
#             new_df = pd.DataFrame(to_150)
#             class_data = pd.concat([class_data, new_df], ignore_index=True)

#     # 2) Now, add an extra 'augmentations_per_class' beyond the *new* count
#     updated_count = len(class_data)
#     final_target = updated_count + augmentations_per_class
#     additional_aug = augment_class_data(class_data, target_total=final_target)
#     augmented_data.extend(additional_aug)

# # ----------------------------
# # 3d. Combine everything
# # ----------------------------
# augmented_df = pd.DataFrame(augmented_data)
# train_df = pd.concat([train_df, augmented_df], ignore_index=True)
print(f"Final augmented training set size: {len(train_df)}")
train_df['product-category'].value_counts()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Final augmented training set size: 6717


,count
product-category,
"meat, egg and dairy products",1433
cereals and bakery products,657
fruits and vegetables,517
prepared dishes and snacks,438
confectionery,325
non-alcoholic beverages,297
"cocoa and cocoa preparations, coffee and tea",296
"soups, broths, sauces and condiments",289
ices and desserts,288


In [ ]:
# Split the data into training and testing sets
# train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create DataCollator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)

# Convert dataset to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/4573 [00:00<?, ? examples/s]

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

* Train

In [ ]:
model_product_category = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))
model_product_category.to('cuda')  # Move model to GPU if available

optimizer = AdamW(model_product_category.parameters(), lr=5e-5)
num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

model_product_category.train()
progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_product_category(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5170 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
# model_product_category = BertForSequenceClassification.from_pretrained("/content/drive/My Drive/FoodHazardData/bert_product_category")
model_product_category = DistilBertForSequenceClassification.from_pretrained("/content/drive/My Drive/FoodHazardData/Models/distilbert_product_category_gemini")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_product_category.to(device)

model_product_category.eval()
total_predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_product_category(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        total_predictions.extend([p.item() for p in predictions])

#print(classification_report(test_df.label, total_predictions, zero_division=0))
predicted_labels = label_encoder.inverse_transform(total_predictions)
gold_labels = label_encoder.inverse_transform(test_df.label.values)
print(classification_report(gold_labels, predicted_labels, zero_division=0))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


                                                   precision    recall  f1-score   support

                              alcoholic beverages       1.00      1.00      1.00         2
                      cereals and bakery products       0.75      0.83      0.79        54
     cocoa and cocoa preparations, coffee and tea       0.81      0.86      0.83        29
                                    confectionery       0.74      0.70      0.72        20
dietetic foods, food supplements, fortified foods       0.82      0.82      0.82        11
                                    fats and oils       1.00      1.00      1.00         3
                                   feed materials       1.00      1.00      1.00         2
                           food contact materials       1.00      1.00      1.00         1
                            fruits and vegetables       0.78      0.77      0.77        56
                                 herbs and spices       0.73      0.89      0.80         

* Test

In [ ]:
model_product_category.save_pretrained("bert_product_category_gemini")
model_product_category.save_pretrained("/content/drive/My Drive/FoodHazardData/Models/bert_product_category_gemini")

# Label: `Hazard`

In [ ]:
label = 'hazard'
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data[label])

# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create DataCollator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)

# Convert dataset to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

In [ ]:
model_hazard = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))
model_hazard.to('cuda')  # Move model to GPU if available

optimizer = AdamW(model_hazard.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

model_hazard.train()

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_hazard(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1527 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
model_hazard.eval()
total_predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_hazard(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        total_predictions.extend([p.item() for p in predictions])

#print(classification_report(test_df.label, total_predictions, zero_division=0))
predicted_labels = label_encoder.inverse_transform(total_predictions)
gold_labels = label_encoder.inverse_transform(test_df.label.values)
print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                                   precision    recall  f1-score   support

                                        Aflatoxin       0.67      0.50      0.57         4
                                  alcohol content       0.00      0.00      0.00         1
                                        alkaloids       0.00      0.00      0.00         2
                                        allergens       0.00      0.00      0.00         4
                                           almond       0.37      0.79      0.50        14
             altered organoleptic characteristics       1.00      0.50      0.67         2
                                        amygdalin       0.00      0.00      0.00         2
                           antibiotics, vet drugs       0.00      0.00      0.00         1
                                    bacillus spp.       0.00      0.00      0.00         1
                             bad smell / off odor       0.00      0.00      0.00         

In [ ]:
model_hazard.save_pretrained("bert_hazard")

# Label: `product`

In [ ]:
label = 'product'
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data[label])

# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create DataCollator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)

# Convert dataset to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

In [ ]:
model_product = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))
model_product.to('cuda')  # Move model to GPU if available

optimizer = AdamW(model_product.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
model_product.train()
progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_product(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1527 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
model_product.eval()
total_predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_product(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        total_predictions.extend([p.item() for p in predictions])

#print(classification_report(test_df.label, total_predictions, zero_division=0))
predicted_labels = label_encoder.inverse_transform(total_predictions)
gold_labels = label_encoder.inverse_transform(test_df.label.values)
print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                                   precision    recall  f1-score   support

                           Catfishes (freshwater)       0.00      0.00      0.00         5
                            Fishes not identified       0.00      0.00      0.00         6
                         Not classified pork meat       0.00      0.00      0.00         3
                       Pangas catfishes (generic)       0.00      0.00      0.00         1
              Precooked cooked pork meat products       0.00      0.00      0.00         1
                                    Veggie Burger       0.00      0.00      0.00         2
                               after dinner mints       0.00      0.00      0.00         1
                                            algae       0.00      0.00      0.00         3
                            all purpose seasoning       0.00      0.00      0.00         1
                                   almond kernels       0.00      0.00      0.00         

In [ ]:
model_product.save_pretrained("bert_product")
tokenizer.save_pretrained("bert_tokenizer")

('bert_tokenizer/tokenizer_config.json',
 'bert_tokenizer/special_tokens_map.json',
 'bert_tokenizer/vocab.txt',
 'bert_tokenizer/added_tokens.json')

In [ ]:
!zip bert_baseline.zip bert_*

  adding: bert_hazard/ (stored 0%)
  adding: bert_hazard_category/ (stored 0%)
  adding: bert_product/ (stored 0%)
  adding: bert_product_category/ (stored 0%)
  adding: bert_tokenizer/ (stored 0%)


# Loading a trained baseline

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

def predict(texts, model_path, tokenizer_path="tokenizer", batch_size=32):
    # Load the saved tokenizer
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

    # Load the saved model
    model = BertForSequenceClassification.from_pretrained(model_path)

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize the input texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Set the model to evaluation mode
    model.eval()

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # for batch in test_dataloader:
        #   batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        #   outputs = model(**batch)
        #   predictions = torch.argmax(outputs.logits, dim=-1)
        #   total_predictions.extend([p.item() for p in predictions])

    return predictions.cpu().numpy().tolist()

    # Initialize an empty list to store predictions
    # all_predictions = []

    # # Process texts in batches
    # for i in range(0, len(texts), batch_size):
    #     batch_texts = texts[i:i + batch_size]  # Create a batch of texts

    #     # Tokenize the input texts for the current batch
    #     inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")

    #     # Move inputs to the same device as the model
    #     inputs = {key: value.to(device) for key, value in inputs.items()}

    #     # Make predictions for the batch
    #     with torch.no_grad():
    #         outputs = model(**inputs)
    #         logits = outputs.logits
    #         predictions = torch.argmax(logits, dim=-1)

    #     # Collect predictions from the current batch
    #     all_predictions.extend(predictions.cpu().numpy().tolist())

    # return all_predictions

In [ ]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

def compute_score(hazards_true, products_true, hazards_pred, products_pred):
  # Convert string labels to numerical labels for hazards
  le_hazards = LabelEncoder()
  hazards_true = le_hazards.fit_transform(hazards_true)

  # compute f1 for hazards:
  f1_hazards = f1_score(
    hazards_true,
    hazards_pred,
    average='macro'
  )

  # Convert string labels to numerical labels for products
  le_products = LabelEncoder()
  products_true = le_products.fit_transform(products_true)

  # compute f1 for products:
  f1_products = f1_score(
    products_true[hazards_pred == hazards_true],
    products_pred[hazards_pred == hazards_true],
    average='macro'
  )

  return (f1_hazards + f1_products) / 2.

In [ ]:
predictions = pd.DataFrame()

### Sub-Task 1:

In [ ]:
import joblib

# Load the pre-fitted LabelEncoders
hazard_label_encoder = joblib.load("hazard-category_label_encoder.pkl")
product_label_encoder = joblib.load("product-category_label_encoder.pkl")

# Store them in a dictionary for easier access if needed
label_encoders = {
    "hazard_category": hazard_label_encoder,
    "product_category": product_label_encoder
}

In [ ]:
for category in ['hazard_category', 'product_category']: #, 'hazard', 'product']:
# for category in ['product_category']:
  c = category.replace('_', '-')
  print(c.upper())
  predictions[category] = predict(test_df.title_text.to_list(), f"/content/drive/My Drive/FoodHazardData/bert_{category}")
  # Decode predictions back to string labels
  label_encoder = label_encoders[category]
  gold = label_encoder.fit_transform(test_df[c])
  print(classification_report(gold, predictions[category], zero_division=0))

HAZARD-CATEGORY


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
print(f"Score Sub-Task 1: {compute_score(test_df['hazard-category'], test_df['product-category'], predictions['hazard_category'], predictions['product_category']):.3f}")
# print(f"Score Sub-Task 2: {compute_score(test_df['hazard'], test_df['product'], predictions['hazard'], predictions['product']):.3f}")

In [ ]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create DataCollator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)

# Convert dataset to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

# Testing

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import pickle

# Load validation data
validation_data = pd.read_csv('/content/drive/My Drive/FoodHazardData/incidents_test.csv', index_col=0)

# Combine title and text for predictions
validation_data['title_text'] = validation_data['title'] + ' ' + validation_data['text']

# Function to tokenize input
def tokenize_function(examples, tokenizer):
    return tokenizer(examples['title_text'], padding=True, truncation=True)

# Function to load model and make predictions
def predict_category(model_path, tokenizer_path, label_encoder_path, validation_data):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    # Tokenize validation data
    dataset = Dataset.from_pandas(validation_data)
    dataset = dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    # Format dataset for PyTorch
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)

    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # Load label encoder
    with open(label_encoder_path, 'rb') as f:
        label_encoder = pickle.load(f)

    # Make predictions
    total_predictions = []
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            total_predictions.extend([p.item() for p in predictions])

    # Decode predictions
    predicted_labels = label_encoder.inverse_transform(total_predictions)
    return predicted_labels

# Paths for hazard-category model, tokenizer, and label encoder
hazard_model_path = "/content/drive/My Drive/FoodHazardData/bert_hazard_category"
hazard_tokenizer_path = "tokenizer"
hazard_label_encoder_path = "hazard-category_label_encoder.pkl"

# Predict hazard-category
validation_data['hazard-category'] = predict_category(
    hazard_model_path,
    hazard_tokenizer_path,
    hazard_label_encoder_path,
    validation_data
)

# Combine hazard-category with title and text for product-category prediction
validation_data['title_text'] = 'Hazard Category: ' + validation_data['hazard-category'] + ' ' + validation_data['title'] + ' ' + validation_data['text']

# Paths for product-category model, tokenizer, and label encoder
product_model_path = "/content/drive/My Drive/FoodHazardData/Models/bert_product_category_gemini"
product_tokenizer_path = "tokenizer"
product_label_encoder_path = "product-category_label_encoder.pkl"

# Predict product-category
validation_data['product-category'] = predict_category(
    product_model_path,
    product_tokenizer_path,
    product_label_encoder_path,
    validation_data
)

# Save to submission.csv
submission = validation_data[['hazard-category', 'product-category']]
submission.to_csv('submission.csv', index=False)

print("Predictions saved to 'submission.csv'")


Map:   0%|          | 0/997 [00:00<?, ? examples/s]

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

Predictions saved to 'submission.csv'


In [ ]:
# download submission as csv
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score
import pickle
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv('food-hazard-detection-semeval-2025.github.io/data/incidents_train.csv', index_col=0)

# Split data into train and test sets
train_df, test_df = train_test_split(data, test_size=0.1, random_state=42)

# Combine title and text for predictions in test set
test_df['title_text'] = test_df['title'] + ' ' + test_df['text']

# Function to tokenize input
def tokenize_function(examples, tokenizer):
    return tokenizer(examples['title_text'], padding=True, truncation=True)

# Function to load model and make predictions
def predict_category(model_path, tokenizer_path, label_encoder_path, validation_data):
    # Load tokenizer
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

    # Tokenize validation data
    dataset = Dataset.from_pandas(validation_data)
    dataset = dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    # Format dataset for PyTorch
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)

    # Load model
    model = BertForSequenceClassification.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # Load label encoder
    with open(label_encoder_path, 'rb') as f:
        label_encoder = pickle.load(f)

    # Make predictions
    total_predictions = []
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            total_predictions.extend([p.item() for p in predictions])

    # Decode predictions
    predicted_labels = label_encoder.inverse_transform(total_predictions)
    return predicted_labels

# Paths for hazard-category model, tokenizer, and label encoder
hazard_model_path = "/content/drive/My Drive/FoodHazardData/bert_hazard_categoryf"
hazard_tokenizer_path = "tokenizer"
hazard_label_encoder_path = "hazard-category_label_encoder.pkl"

# Predict hazard-category
test_df['hazard-category-pred'] = predict_category(
    hazard_model_path,
    hazard_tokenizer_path,
    hazard_label_encoder_path,
    test_df
)

# Combine hazard-category with title and text for product-category prediction
test_df['title_text'] = 'Hazard Category: ' + test_df['hazard-category'] + ' ' + test_df['title'] + ' ' + test_df['text']

# Paths for product-category model, tokenizer, and label encoder
product_model_path = "/content/drive/My Drive/FoodHazardData/bert_product_category3"
product_tokenizer_path = "tokenizer"
product_label_encoder_path = "product-category_label_encoder.pkl"

# Predict product-category
test_df['product-category-pred'] = predict_category(
    product_model_path,
    product_tokenizer_path,
    product_label_encoder_path,
    test_df
)

# Compute classification report and final score
def compute_score(hazards_true, products_true, hazards_pred, products_pred):
    # Compute F1 for hazards
    f1_hazards = f1_score(hazards_true, hazards_pred, average='macro')

    # Compute F1 for products (only for correctly predicted hazards)
    correct_hazards = hazards_true == hazards_pred
    f1_products = f1_score(
        products_true[correct_hazards],
        products_pred[correct_hazards],
        average='macro'
    )

    return (f1_hazards + f1_products) / 2

# Extract true and predicted labels
hazards_true = test_df['hazard-category']
products_true = test_df['product-category']
hazards_pred = test_df['hazard-category-pred']
products_pred = test_df['product-category-pred']

# Classification report
print("Classification Report for Hazard-Category:")
print(classification_report(hazards_true, hazards_pred))

print("Classification Report for Product-Category (correct hazards only):")
correct_hazards = hazards_true == hazards_pred
print(classification_report(products_true[correct_hazards], products_pred[correct_hazards]))

# Compute final score
final_score = compute_score(hazards_true, products_true, hazards_pred, products_pred)
print(f"Final Score: {final_score}")


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

Classification Report for Hazard-Category:
                                precision    recall  f1-score   support

                     allergens       0.96      0.99      0.98       188
                    biological       0.99      0.99      0.99       171
                      chemical       0.94      0.94      0.94        35
food additives and flavourings       0.80      0.80      0.80         5
                foreign bodies       1.00      0.98      0.99        58
                         fraud       0.92      0.79      0.85        28
                     migration       1.00      1.00      1.00         1
          organoleptic aspects       0.75      1.00      0.86         3
                  other hazard       0.85      0.73      0.79        15
              packaging defect       0.75      0.60      0.67         5

                      accuracy                           0.96       509
                     macro avg       0.90      0.88      0.89       509
                  w

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
from sklearn.metrics import f1_score

def compute_score(hazards_true, products_true, hazards_pred, products_pred):
    """
    Computes the macro-F1 for hazards, then the macro-F1 for products
    *only for rows where hazards_pred == hazards_true*.
    Finally returns the average of those two scores.
    """
    # -- F1 for hazards (macro) --
    f1_hazards = f1_score(hazards_true, hazards_pred, average='macro')

    # -- F1 for products, only on rows where hazard is correct --
    correct_hazard_mask = (hazards_pred == hazards_true)
    # If no hazards were correct, this can raise an error.
    # For safety, you might do a check to handle that edge case.
    f1_products = f1_score(
        products_true[correct_hazard_mask],
        products_pred[correct_hazard_mask],
        average='macro'
    )

    return (f1_hazards + f1_products) / 2.0

# -----------------------------
# 1. Read the ground truth
# -----------------------------
df_valid = pd.read_csv("incidents_valid.csv")

# We only need these columns from the ground truth
df_valid = df_valid[["hazard-category", "product-category"]].copy()
df_valid.columns = ["hazard_true", "product_true"]

# -----------------------------
# 2. Read your submission
# -----------------------------
df_sub = pd.read_csv("submission.csv")

# We only need these columns from the submission
df_sub = df_sub[["hazard-category", "product-category"]].copy()
df_sub.columns = ["hazard_pred", "product_pred"]

# -----------------------------
# 3. Concatenate or merge to compare row by row
# -----------------------------
# *** IMPORTANT ***
# This assumes that the number of rows in df_valid
# and df_sub match, and that each row corresponds
# to the same incident.
df_compare = pd.concat([df_valid, df_sub], axis=1)

# -----------------------------
# 4. Compute final score
# -----------------------------
score = compute_score(
    hazards_true=df_compare["hazard_true"],
    products_true=df_compare["product_true"],
    hazards_pred=df_compare["hazard_pred"],
    products_pred=df_compare["product_pred"]
)

print("Your macro-F1 Score (with the given formula) =", score)


Your macro-F1 Score (with the given formula) = 0.8107705657583926


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

# Load the ground truth and submission data
df_valid = pd.read_csv("incidents_valid.csv")
df_sub = pd.read_csv("submission.csv")

# Rename columns for consistency
df_valid = df_valid.rename(columns={"hazard-category": "hazard_true", "product-category": "product_true"})
df_sub = df_sub.rename(columns={"hazard-category": "hazard_pred", "product-category": "product_pred"})

# Concatenate the two dataframes for comparison
df_compare = pd.concat([df_valid, df_sub], axis=1)

# Filter rows where `hazard-category` is correctly predicted
correct_hazard_rows = df_compare[df_compare["hazard_true"] == df_compare["hazard_pred"]]

# Get true and predicted labels for `product-category`
product_true = correct_hazard_rows["product_true"]
product_pred = correct_hazard_rows["product_pred"]

# Print the classification report
print(classification_report(product_true, product_pred, zero_division=0))

                                                   precision    recall  f1-score   support

                              alcoholic beverages       1.00      1.00      1.00         4
                      cereals and bakery products       0.79      0.76      0.77        72
     cocoa and cocoa preparations, coffee and tea       0.73      0.73      0.73        15
                                    confectionery       0.79      0.60      0.68        25
dietetic foods, food supplements, fortified foods       0.53      0.71      0.61        14
                                    fats and oils       0.75      1.00      0.86         3
                            fruits and vegetables       0.74      0.78      0.76        51
                                 herbs and spices       0.65      0.79      0.71        19
                                ices and desserts       0.80      0.87      0.83        23
                     meat, egg and dairy products       0.88      0.82      0.85       13

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

# Load the ground truth and submission data
df_valid = pd.read_csv("incidents_valid.csv")
df_sub = pd.read_csv("submission.csv")

# Print the classification report
print(classification_report(df_valid['hazard-category'], df_sub['hazard-category'], zero_division=0))

                                precision    recall  f1-score   support

                     allergens       0.95      1.00      0.97       207
                    biological       0.98      0.99      0.99       194
                      chemical       0.90      0.96      0.93        28
food additives and flavourings       1.00      1.00      1.00         2
                foreign bodies       0.98      0.98      0.98        63
                         fraud       0.88      0.73      0.80        41
          organoleptic aspects       0.86      0.75      0.80         8
                  other hazard       0.75      0.64      0.69        14
              packaging defect       0.83      0.62      0.71         8

                      accuracy                           0.95       565
                     macro avg       0.91      0.85      0.88       565
                  weighted avg       0.95      0.95      0.95       565



In [ ]:
import os
from shutil import make_archive

# save predictions to a new folder:
os.makedirs('./submission/', exist_ok=True)
predictions.to_csv('./submission/submission.csv')

# zip the folder (zipfile can be directly uploaded to codalab):
make_archive('./submission', 'zip', './submission')

'/content/submission.zip'